## HGN Model Evaluation

In [1]:
import pickle
import math
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

dir = 'E:\Sebnewrepo\Rec_sys_lab\HGN/'
user_record_file = 'Books_item_sequences.pkl'
user_mapping_file = 'Books_user_mapping.pkl'
item_mapping_file = 'Books_item_mapping.pkl'

#### Import HGN Model and Evaluation Function

In [2]:
from gating_network import HGN
from interactions import Interactions
from eval_metrics import *

import argparse
import logging
from time import time
import datetime
import torch

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

In [3]:
#check the device if available
torch.cuda.is_available()

True

In [4]:
def evaluation(hgn, train, test_set, topk=20):
    num_users = train.num_users
    num_items = train.num_items
    batch_size = 1024
    num_batches = int(num_users / batch_size) + 1
    user_indexes = np.arange(num_users)
    item_indexes = np.arange(num_items)
    pred_list = None
    train_matrix = train.tocsr()
    test_sequences = train.test_sequences.sequences

    for batchID in range(num_batches):
        start = batchID * batch_size
        end = start + batch_size

        if batchID == num_batches - 1:
            if start < num_users:
                end = num_users
            else:
                break

        batch_user_index = user_indexes[start:end]

        batch_test_sequences = test_sequences[batch_user_index]
        batch_test_sequences = np.atleast_2d(batch_test_sequences)

        batch_test_sequences = torch.from_numpy(batch_test_sequences).type(torch.LongTensor).to(device)
        item_ids = torch.from_numpy(item_indexes).type(torch.LongTensor).to(device)
        batch_user_ids = torch.from_numpy(np.array(batch_user_index)).type(torch.LongTensor).to(device)

        rating_pred = hgn(batch_test_sequences, batch_user_ids, item_ids, True)
        rating_pred = rating_pred.cpu().data.numpy().copy()
        rating_pred[train_matrix[batch_user_index].toarray() > 0] = 0

        # reference: https://stackoverflow.com/a/23734295, https://stackoverflow.com/a/20104162
        ind = np.argpartition(rating_pred, -topk)
        ind = ind[:, -topk:]
        arr_ind = rating_pred[np.arange(len(rating_pred))[:, None], ind]
        arr_ind_argsort = np.argsort(arr_ind)[np.arange(len(rating_pred)), ::-1]
        batch_pred_list = ind[np.arange(len(rating_pred))[:, None], arr_ind_argsort]

        if batchID == 0:
            pred_list = batch_pred_list
        else:
            pred_list = np.append(pred_list, batch_pred_list, axis=0)

    precision, recall, MAP, ndcg = [], [], [], []
    for k in [5, 10, 15, 20]:
        precision.append(precision_at_k(test_set, pred_list, k))
        recall.append(recall_at_k(test_set, pred_list, k))
        MAP.append(mapk(test_set, pred_list, k))
        ndcg.append(ndcg_k(test_set, pred_list, k))

    return precision, recall, MAP, ndcg


def negsamp_vectorized_bsearch_preverif(pos_inds, n_items, n_samp=32):
    """ Pre-verified with binary search
    `pos_inds` is assumed to be ordered
    reference: https://tech.hbc.com/2018-03-23-negative-sampling-in-numpy.html
    """
    raw_samp = np.random.randint(0, n_items - len(pos_inds), size=n_samp)
    pos_inds_adj = pos_inds - np.arange(len(pos_inds))
    neg_inds = raw_samp + np.searchsorted(pos_inds_adj, raw_samp, side='right')
    return neg_inds


def generate_negative_samples(train_matrix, num_neg=3, num_sets=10):
    neg_samples = []
    for user_id, row in enumerate(train_matrix):
        pos_ind = row.indices
        neg_sample = negsamp_vectorized_bsearch_preverif(pos_ind, train_matrix.shape[1], num_neg * num_sets)
        neg_samples.append(neg_sample)

    return np.asarray(neg_samples).reshape(num_sets, train_matrix.shape[0], num_neg)


def train_model(train_data, test_data, config):
    num_users = train_data.num_users
    num_items = train_data.num_items

    # convert to sequences, targets and users
    sequences_np = train_data.sequences.sequences
    print(sequences_np)
    targets_np = train_data.sequences.targets
    users_np = train_data.sequences.user_ids
    train_matrix = train_data.tocsr()

    n_train = sequences_np.shape[0]
    logger.info("Total training records:{}".format(n_train))

    hgn = HGN(num_users, num_items, config, device).to(device)

    optimizer = torch.optim.Adam(hgn.parameters(), lr=config.learning_rate, weight_decay=config.l2)

    record_indexes = np.arange(n_train)
    batch_size = config.batch_size
    num_batches = int(n_train / batch_size) + 1
    for epoch_num in range(config.n_iter):

        t1 = time()

        # set model to training mode
        hgn.train()

        np.random.shuffle(record_indexes)

        t_neg_start = time()
        negatives_np_multi = generate_negative_samples(train_matrix, config.neg_samples, config.sets_of_neg_samples)
        logger.info("Negative sampling time: {}s".format(time() - t_neg_start))

        epoch_loss = 0.0
        for batchID in range(num_batches):
            start = batchID * batch_size
            end = start + batch_size

            if batchID == num_batches - 1:
                if start < n_train:
                    end = n_train
                else:
                    break

            batch_record_index = record_indexes[start:end]

            batch_users = users_np[batch_record_index]
            batch_sequences = sequences_np[batch_record_index]
            batch_targets = targets_np[batch_record_index]
            negatives_np = negatives_np_multi[batchID % config.sets_of_neg_samples]
            batch_neg = negatives_np[batch_users]

            batch_users = torch.from_numpy(batch_users).type(torch.LongTensor).to(device)
            batch_sequences = torch.from_numpy(batch_sequences).type(torch.LongTensor).to(device)
            batch_targets = torch.from_numpy(batch_targets).type(torch.LongTensor).to(device)
            batch_negatives = torch.from_numpy(batch_neg).type(torch.LongTensor).to(device)

            items_to_predict = torch.cat((batch_targets, batch_negatives), 1)
            prediction_score = hgn(batch_sequences, batch_users, items_to_predict, False)

            (targets_prediction, negatives_prediction) = torch.split(
                prediction_score, [batch_targets.size(1), batch_negatives.size(1)], dim=1)

            # compute the BPR loss
            loss = -torch.log(torch.sigmoid(targets_prediction - negatives_prediction) + 1e-8)
            loss = torch.mean(torch.sum(loss))

            epoch_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # clean the grad, 
            #optimizer.zero_grad()
        epoch_loss /= num_batches

        t2 = time()

        output_str = "Epoch %d [%.1f s]  loss=%.4f" % (epoch_num + 1, t2 - t1, epoch_loss)
        logger.info(output_str)

        if (epoch_num + 1) % 20 == 0:
            hgn.eval()
            precision, recall, MAP, ndcg = evaluation(hgn, train_data, test_data, topk=20)
            logger.info(', '.join(str(e) for e in precision))
            logger.info(', '.join(str(e) for e in recall))
            logger.info(', '.join(str(e) for e in MAP))
            logger.info(', '.join(str(e) for e in ndcg))
            logger.info("Evaluation time:{}".format(time() - t2))
    logger.info("\n")
    logger.info("\n")
    

#### Load dataset and split train-test sets

In [5]:
def load_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f, encoding='latin1')

# split train test data        
def split_data_sequentially(user_records, test_radio=0.2):
    train_set = []
    test_set = []

    for item_list in user_records:
        len_list = len(item_list)
        num_test_samples = int(math.ceil(len_list * test_radio))
        train_sample = []
        test_sample = []
        for i in range(len_list - num_test_samples, len_list):
            test_sample.append(item_list[i])
            
        for place in item_list:
            if place not in set(test_sample):
                train_sample.append(place)
                
        train_set.append(train_sample)
        test_set.append(test_sample)

    return train_set, test_set
    
def data_index_shift(lists, increase_by):
    '''
    Increase the item index to contain the pad_index
    :param lists:
    :param increase_by: 1
    :return:
    '''
    
    for seq in lists:
        for i, item_id in enumerate(seq):
            seq[i] = item_id + increase_by

    return lists

def generate_dataset(index_shift=1):
    user_records = load_pickle(dir + user_record_file)
    user_mapping = load_pickle(dir + user_mapping_file)
    item_mapping = load_pickle(dir + item_mapping_file)
        
    # validate the input data
    #assert len(user_mapping) == 52406 and len(item_mapping) == 41264
    # apply small dataset
    #user_records = user_records[0:10000]
    #user_mapping = user_mapping[0:10000]
    
    user_records = data_index_shift(user_records, increase_by=index_shift)

    # split dataset
    train_val_set, test_set = split_data_sequentially(user_records, test_radio=0.2)
    train_set, val_set = split_data_sequentially(train_val_set, test_radio=0.1)

    return train_set, val_set, train_val_set, test_set, len(user_mapping), len(item_mapping) + index_shift

In [6]:
# get train test dataset
# item_id=0 for sequence padding
train_set, val_set, train_val_set, test_set, num_users, num_items = generate_dataset(index_shift = 1)

In [7]:
parser = argparse.ArgumentParser()

# data arguments
parser.add_argument('--L', type=int, default=5)
parser.add_argument('--T', type=int, default=3)
# train arguments
parser.add_argument('--n_iter', type=int, default=200)
parser.add_argument('--seed', type=int, default=1234)
parser.add_argument('--batch_size', type=int, default=4096)
parser.add_argument('--learning_rate', type=float, default=1e-3)
parser.add_argument('--l2', type=float, default=1e-3)
parser.add_argument('--neg_samples', type=int, default=3)
parser.add_argument('--sets_of_neg_samples', type=int, default=50)

# model dependent arguments
parser.add_argument('--d', type=int, default=50)
config = parser.parse_args(
    args = [
        '--L', '5',
        '--T', '3',
        '--n_iter', '1',
        '--seed', '1200',
        '--batch_size', '4000',
        '--learning_rate', '0.001',
        '--l2', '0.001',
        '--neg_samples', '3',
        '--sets_of_neg_samples', '50'
    ])

In [8]:
train = Interactions(train_val_set, num_users, num_items)

In [9]:
train.num_users

52406

In [10]:

train = Interactions(train_val_set, num_users, num_items)
train.to_sequence(config.L, config.T)

logger.info(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
logger.info(config)
train_model(train, test_set, config)

INFO:__main__:2021-05-23 12:04:20
INFO:__main__:Namespace(L=5, T=3, batch_size=4000, d=50, l2=0.001, learning_rate=0.001, n_iter=1, neg_samples=3, seed=1200, sets_of_neg_samples=50)
INFO:__main__:Total training records:1097612


[[30761 24287 31427 25107 25129]
 [21242 21014 12180 16462 25352]
 [24740 21242 21014 12180 16462]
 ...
 [29020 18596 13589 12496 38343]
 [39305 29020 18596 13589 12496]
 [31673 39305 29020 18596 13589]]


INFO:__main__:Negative sampling time: 2.690805435180664s
INFO:__main__:Epoch 1 [6.2 s]  loss=6412.2515
INFO:__main__:

INFO:__main__:

